# Libreria standart: Pickle

**NOTA: Los archivos pickle NO son seguros, solo debemos deserializar pickle que sepamos que son seguros, los de nuestro equipo o de una fuente fiable.**

Este módulo nos permite **serializar** un objeto Python en un archivo binario (archivo pickle). Este archivo binario puede almacenarse, y despues puede **deserializarse** para recuperar el objeto en cuestión.

Serializar es el nombre técnico para describir un algoritmo inespecifico que transforma un tipo en otro (generalmente binario o texto).

## Codificación y decodificación

```python
from pickle import dumps

diccionario = {"a": "A", "b": 1.0, "c": (1, 2)}
mi_pickle = dumps(diccionario)  # Transforma objeto Python a pickle, un objeto tipo-archivo
```

`dumps()` serializa a binario un objeto Python. *En general, como queremos trabajar con un tipo-archivo, no se utilizará esta función*.

```python
from pickle import loads

pickle_a_python = loads(mi_pickle)  # Crea un nuevo objeto igual que el almacenado en el pickle
```

`loads()` deserializa **un string binario** en objeto Python.

```python
from pickle import dump

diccionario = {"a": "A", "b": 1.0, "c": (1, 2)}
with open("mi_pickle.pickle", "wb") as archivo:
    dump(diccionario, archivo)  # Serializa en binario a un tipo-archivo y lo guarda como archivo .pickle
```

`dump()` es exactamente igual que `dumps()`, serializa objeto Python a tipo-archivo **y permite añadirlo a un buffer o gestor I/O directamente**.

```python
from pickle import load

with open("mi_pickle.pickle", "rb") as archivo:
    diccionario_reconstruido = load(archivo)
```

`load()` deserializa **un tipo-archivo** en objeto Python.

## Objetos que no pueden usarse con pickle

No funciona bien para clases, ya que esta diseñado para instancias en vez de clases. Además, objetos con un **estado en runtime** como buffers, sockets, gestores de archivos, streams, ... que dependen de procesos o estan activos *no pueden usarse con pickle*.

## Referencias circulares

`pickle` automaticamente maneja las referencias circulares.

# Libreria standart: JSON

Un JSON (JavaScript Object Notation) es un "diccionario" de JavaScript que representa un objeto en ese lenguaje. Al igual que `pickle`, es un *sistema de serialización*, en este caso universal (`pickle` solo lo tenemos en Python. Y en vez de binario, la serialización es en texto.

JSON se utiliza especialmente en la web, ya que toda comunicación HTTP utiliza binario o texto, siendo el texto un JSON o un HTML completo.

## Codificación y decodificación

```python
from json import dumps

diccionario = {"a": "A", "b": 1.0, "c": (1, 2)}
mi_json = dumps(diccionario)  # Transforma objeto Python a JSON
```

`dumps()` serializa objeto Python a JSON. La serialización es a tipo `str` y la función tiene varias kwargs para formatear el JSON. La kwarg `sort_keys=<bool>` permite ordenar el JSON, `indent=<int>` especifica el nivel de indentación (util en JSON muy anidados), `separators=<tuple[str]>` personaliza los separadores, ... pero estas kwargs *no se deberian de utilizar en producción* ya que consumen recursos innecesariamente.

Los JSON **no admiten un tipo distinto a `str` como clave**, por lo que utilizar un numero como clave producirá un error. `skipkeys=True` permite saltar estas keywords y no incorporarlas al JSON

```python
from json import loads

de_json_a_diccionario = loads(mi_json)  # Transforma JSON a objeto Python
```

`loads()` decodifica un JSON en un objeto Python. El JSON puede estar en texto (`str`), `bytes` o `bytearray`. Tiene varias kwargs para manejar la decodificación y una de ellas permite hacer un callback a un hook. Este callback es especialmente interesante, ya que permite **parsear de un JSON a un objeto arbitrario**, entre otras cosas, definiendo nosotros una *función dict_a_object()* (librerias como Pydantic basan toda su funcionalidad en esto!). Hay un hook para funciones, ints, floats y constantes.

```python
from json import dump
import io

datos = {"a": "A", "b": 1.0, "c": (1, 2)}
archivo = io.StringIO()  # Buffer de texto que permite utilizar este texto como archivo, stream o socket
mi_json = dump(datos, archivo)  # Transforma objeto Python a JSON añadiendolo a un buffer, que podemos utilizar de multiples formas
```

`dump()` serializa objeto Python a JSON **y añade a un buffer o archivo tipo-archivo**. Tiene los mismos kwargs que `dumps()`.

```python
from json import load

with open("mi_json.json") as archivo:
    mi_json = load(archivo)  # Lee un tipo-archivo (en este caso un buffer con un archivo en disco)
```

`load()` decodifica un JSON a objeto Python **desde un buffer o tipo-archivo**. Tiene los mismos kwargs que `loads()`.

# Iteración: Protocolo e iteradores

El *patrón iterador* es un **patrón de diseño** muy utilizado, y su implementación en Python es excelente. Python tiene un **protocolo de iteración** que se aplica a los **iteradores**, objetos que pueden *iterarse* (es decir, que provienen de un **iterable**). Un iterador es un objeto con dos métodos especiales: *next* y *done*.

Python implementa *next* como **__next__()**, e incluye una función para abstraer este método, `next()`. Y *no incluye un done*, en vez de esto, hace un `raise` de `StopIteration`. La consecuencia de esto es que **se tiene un gestor de contexto de la forma `for item in iterable`**.

Debido a la interfaz ABC de los iteradores, un iterador debe incluir tambien un método **__iter__()**, que se suele definir de forma explicita o implicita en la clase iterable, de forma que se pueda crear un iterador con la funcion `iter()`.

In [ ]:
# Iterable hecho a mano
from typing import Iterator

class Frase:
    def __init__(self, texto: str):
        self.frase = texto
        self.palabras = texto.split()  # Esto divide una frase en una lista de palabras, que le daremos al iterador
        
    def __iter__(self) -> Iterator:
        return IteradorFrases(self.palabras)  # Esto permite crear un iterador desde este iterable

# Iterador hecho a mano
class IteradorFrases:
    def __init__(self, palabras: list[str]):
        self.palabras = palabras  # Las palabras que tenemos en la lista de palabras
        self.indice = 0  # Indice para la iteracion

    def __next__(self) -> str:
        try:
            palabra = self.palabras[self.indice]  # Vamos iterando palabra por palabra
        except IndexError:  # Si hemos recorrido el iterable entero, lanzamos un StopIteration para terminar la iteracion
            raise StopIteration()
        self.indice += 1  # Subimos uno el indice para que podamos iterar
        return palabra.upper()  # Para demostrar el iterable, estoy poniendo las palabras en mayuscula

    def __iter__(self) -> Iterator:  # Esto no es necesario, pero permite cumplir con la ABC de los iteradores
        return self

# Probamos la iteracion a mano
frase = Frase("esto es una frase en mayusculas!")  # Definimos el iterable
iterador_frase = iter(frase)  # Creamos el iterador
try:
    print(next(iterador_frase))
    print(next(iterador_frase))
    print(next(iterador_frase))
    print(next(iterador_frase))
    print(next(iterador_frase))
    print(next(iterador_frase))
    print(next(iterador_frase))
    print(next(iterador_frase))
except StopIteration:
    print("Se ha parado la iteracion (raise StopIteration)\n")

# Probamos la iteracion en un context manager
fr = Frase("mira como itero esta frase")
for palabra in fr:
    print(palabra)

# Generadores

**El hecho de que podamos iterar cada elemento individualmente cada vez, hace que podamos ir considerando ese mismo elemento cada vez que lo visitamos**. Este es el fundamento de los generadores. Algunas veces tenemos demasiados datos para tener en memoria, o queremos ahorrar **mucho** tiempo al iterar, o al leer, o mandar datos. Para eso se crearon los **generadores**.

Un generador es un *objeto que va devolviendo los objetos que va iterando, uno cada vez*, y **se va consumiendo por cada objeto que se visita**. Se utilizan con keyword `yield`.

In [ ]:
# Mismo ejemplo de antes pero con un generador
class Frase:
    def __init__(self, texto: str):
        self.frase = texto
        self.palabras = texto.split()
        
    def __iter__(self) -> str:  # Ya no me devuelve un Iterator
        for palabra in self.palabras:
            yield palabra.upper()  # Ahora esta clase es un generador, que me va devolviendo cada palabra en mayusculas

# Iteracion a mano

frase = Frase("esto es una frase en mayusculas!")
iterador_frase = iter(frase)
print(next(iterador_frase))
print(next(iterador_frase))

# Iteracion con bucle
frase = Frase("esto es un test")
for palabra in frase:
    print(palabra)

Generalmente, los generadores se *definen como funciones*.

In [ ]:
# Un generador para numeros Fibonacci
from typing import Generator

def generador_fibonacci() -> Generator[int]:
    numero1, numero2 = 0, 1
    while True:  # Iteracion infinita
        yield numero1
        numero1, numero2 = numero2, numero1 + numero2

fibonacci = generador_fibonacci()
for _ in range(10):  # Imprime los 10 primeros numeros Fibonacci
    print(next(fibonacci))



# Un generador mas elegante

def generador_fibonacci(numeros_a_crear: int) -> Generator[str, int]:
    numero1, numero2 = 0, 1
    for _ in range(numeros_a_crear):  # range() es otro generador, pudiendo utilziar generadores en generadores
        yield "Siguiente numero" if numero1 != 0 else ""  # Un generador puede tener varios yield
        yield numero1
        numero1, numero2 = numero2, numero1 + numero2

numeros = 10  # Número de números de Fibonacci a generar
fibonacci = generador_fibonacci(numeros)
for numero in fibonacci:
    print(numero)

# Comprensiones

Las **comprensiones** son generadores abstraidos que nos permiten *realizar iteraciones y crear a la vez una variable* utilizando un generador anonimo. Son muy cómodas de utilizar y muy eficientes, y **reemplazan por completo los medios tradicionales de programación funcional, como map, filter o reduce**.

```python
var = ( mapeo(<elemento>) for <elemento> in <iterable> if <condicion> )
```

La comprensión mas básica es una comprensión de generador.

In [ ]:
# Ejemplo comprension generador

cuadrados = (x**2 for x in range(1, 11))  # Esto crea un generador
for numero in cuadrados:
    print(numero)

## Comprensiones para todas las colecciones

**Toda comprensión utiliza por debajo un generador**, y por tanto, utilizan en realidad una comprensión de generador.

* Comprension de lista

```python
frase = "esto es un test"
palabras_mayuscula = [palabra.upper() for palabra in frase.split()]
```

* Comprension de set

```python
numeros = 1, 6, 2, 7, 3, 9, 4, 1, 2, 7, 3, 5, 1, 7, 2, 8, 3, 5, 6, 8, 3, 2, 7
numeros_menores_5 = {numero for numero in numeros if numero < 5}
```

* Comprension de diccionario

```python
fechas = "01-04-1994,13-02-1999,22-10-2002,14-08-2000"
usuarios = ["Joaquin", "Rodrigo", "Lorenzo", "Beatriz"]
diccionario = {_id: (fecha, usuario) for _id, fecha, usuario in zip(range(1,5), usuarios, fechas)}
```

Además, las comprensiones se pueden **anidar**.

```python
# El orden es de derecha a izquierda, por lo que el segundo numero ira iterandose primero
generador_anidado = ((primer_numero, segundo_numero) for primer_numero in range(1, 4) for segundo_numero in range(1, 4))
```

# Libreria standart: Itertools

Este módulo nos permite trabajar con iteradores de formas mucho más complejas y holisticas. Incluye funciones rápidas y eficientes, que pueden encadenarse para llevar el patrón iterador al siguiente nivel. Ingenieros senior en análisis de datos e ingenería de datos suelen manejar con soltura este módulo.

```python
from itertools import chain

iterable1, iterable2, iterable3 = [1, 2, 3], ["a", "b", "c"], (numero + 10 for numero in range(3)
for elemento in chain(iterable1, iterable2, iterable3):
    ...

def iterables():
    yield [1, 2, 3]
    yield ["a", "b", "c"]
    yield (numero + 10 for numero in range(3)

for elemento in chain.from_iterable(iterables()):
    ...
```

`chain()` concatena iteradores, como si todos fueran parte de un único iterador más grande. `.from_iterable(<generador>)` permite hacerlo desde un generador.

```python
from itertools import zip_longest

iterable1, iterable2, iterable3 = [1, 2, 3], ["a", "b", "c"], (numero + 10 for numero in range(10))
zip_normal = zip(iterable1, iterable2, iterable3)
zip_largo = zip_longest(iterable1, iterable2, iterable3)

print(zip_normal)
print(zip_longest)
```

`zip_longest()` crea tuplas de los iterables, igual que `zip()`, pero continua hasta el iterable más largo, y sustituye los valores que no existen por `None`.

```python
from itertools import islice

lista_larga = list(range(100))
donde_quiero_empezar: int = 5
donde_quiero_parar: int = 20
salto = 2
slice_por_posicion = islice(lista_larga, donde_quiero_empezar, donde_quiero_parar, salto)
```

`islice()` tiene la sintaxis de un slice normal [inicio:final:salto]. Si solo se le da un iterable y un int, devuelve hasta ese indice.

```python
from itertools import islice, tee

trozo = islice(range(100), 5
iter1, iter2 = tee(trozo)
```

`tee()` crea iteradores independientes (como mínimo 2) para un iterador. Muy parecido a `tee` de Linux.

```python
from itertools import starmap

numeros_multiplicacion = [(0, 5), (1, 3), (3, 5), (7, 4)]
funcion_multiplicacion = lambda n1, n2: (n1, n2, n1 * n2)
mapeo = starmap(funcion_multiplicacion, numeros_multiplicacion)
for multiplicacion in mapeo:
    print(*multiplicacion)  # Podemos desempaquetar con * gracias a que hemos usado starmap
```

`starmap()`es como `map()` pero desempaqueta sus elementos con `*` separando asi los iteradores. Ejemplo:

```python
from itertools import starmap

lista = [(1, 5), (1, 3), (3, 5), (7, 4)]
lista_exponencial = list(map(pow, lista))  # Con map, pow no podria usarse porque requiere de dos argumentos
lista_exponencial = list(starmap(pow, lista))  # starmap rellena pow(<1>, <2>) desempaquetando las tuplas
```

```python
from itertools import count

i = 10
while i < 20:
    print(count(i))  # Si no lo controlamos, es infinito!!
```

`count()` es como `range()` pero, al ser un generador, es *infinito*. Construye int consecutivos. Se le puede definir un step arbitrario.

```python
from itertools import cycle

anillo = ["a", "b", "c"]
for combinacion in zip(range(10), cycle(anillo)):
    print(combinacion)
```

`cycle()`itera de forma cíclica un iterable.

```python
from itertools import repeat

ejemplo = "lo vamos a repetir"
for i in repeat(ejemplo):
    print(i)
```

`repeat()`repite in objeto de forma infinita, a menos que se le especifique un valor de parada.

```python
from itertools import accumulate
import operator

lista = [2, 3, 4, 5]
result = accumulate(lista, operator.mul)
for acumulacion in result:
    print("acumulacion")
```

`accumulate()` va acumulando poco a poco un iterable de acuerdo a una operacion.

```python
from itertools import takewhile, dropwhile

data = list(range(11)) + [1, 2, 3]
menores_5 = takewhile(lambda numero: numero < 5, data)
mayores_7 = dropwhile(lambda numero: numero <= 7, data)
```

`takewhile()` evalua elemento a elemento y los incluye en un nuevo iterable *cuando la condición se cumpla*. `dropwhile()` hace lo mismo pero, en vez de incluirlos, los excluye.

```python
from itertools import combinations

pelotas = ("roja", "verde", "azul")
eleccion = combinations(pelotas, 3)
```

`combinations()` crea una tupla para cada posible combinación de los elementos de un iterable.

```python
from itertools import combinations_with_replacement

pelotas = ("roja", "verde", "azul")
eleccion = combinations_with_replacement(pelotas, 3)
```

`combinations_with_replacement()` crea una tupla para cada posible combinación de los elementos de un iterable, con repetición de estos elementos.

```python
from itertools import permutations

pelotas = ("roja", "verde", "azul")
eleccion = permutations(pelotas, 3)
```

`permutations()` crea una tupla para cada posible permutación de los elementos de un iterable.

```python
from itertools import product

eje_x = [1, 2, 3]
eje_y = [10, 20, 30]
producto_cartesiano = product(eje_x, eje_y)
```

`product()` crea una tupla para cada multiplicacion cartesiana entre dos iterables.

```python
from itertools import compress

seleccion = (True, False, False, True, False)
colores = ["rojo", "verde", "amarillo", "azul", "negro"]
colores_elegidos = compress(colores, seleccion)
```

`compress()` filtra un iterable de acuerdo a condiciones lógicas de otro iterable (filtra un iterable con otro).

```python
from itertools import groupby

alumnos = [
    {"nombre": "Hector", "clase": "economia"},
    {"nombre": "Joaquin", "clase": "artes"},
    {"nombre": "Irina", "clase": "artes"},
    {"nombre": "Laura", "clase": "matematicas"},
    {"nombre": "Carlos", "clase": "matematicas"},
    {"nombre": "Teresa", "clase": "matematicas"},
    {"nombre": "Roland", "clase": "economia"}
]
agrupacion = groupby(alumnos, key=lambda alumno: alumno["clase"])
for clase, alumnos in agrupacion:
    print(f"{clase}: {list(alumnos)}")
```

`groupby()` agrupa un iterable de acuerdo a una llave, parecido a las llaves de sorted() pero en vez de ordenar, agrupa.